# **Ayudantía 8: Networking** 🖥️


Autores 👨‍💻 

* Diego Milla [@DiegoMillaUC]
* Gustavo Truan [@Gtruaan]

### ¿Qué es Computer Networking?

Dispositivos computacionales interconectados, de forma física o inalámbrica, que pueden intercambiar datos y compartir recursos entre sí, en base a un sistema de reglas llamadas protocolos de comunicación.

## Arquitectura Cliente - Servidor 🤝

### ¿Para qué?
El establecer roles en la comunicación permite un uso eficiente de los recursos y robustez del sistema.

<center><img src="./imagenes/networking.png"/></center>

### Roles

#### Servidor
* Se encuentra a la espera de recibir y manejar conexiones con clientes.
* Entrega servicios a consumir.
* Se encarga del manejo de información, tanto en su fondo como en su forma.

#### Cliente
* Entabla comunicación con el Servidor.
* Realiza solicitudes y consume servicios.

## Sockets

### ¿Qué es?

Es un objeto del sistema operativo que permite a un programa transmitir y recibir datos desde y hacia otro programa.

### ¿Cómo se usa?

#### Cliente TCP (Transmission Control Protocol)

Para crear un socket, se debe crear una instancia de la clase `socket(family, type)`.
* `family`: tipo de dirección IP a usar.
    * Para direcciones IPv4: `AF_INET`.
    * Para direcciones IPv6: `AF_INET6`.
* `type`: protocolo de transporte a usar.
    * Para protocolo TCP: `SOCK_STREAM`.
    * Para protocolo UDP: `SOCK_DGRAM`.

Para conectarse a un servidor TCP se usa el método `connect((host, port))` del socket. 
* `host`: dirección IP del host destinatario.
* `port`: puerto del host destinatario.

Para enviar un mensaje, se usa el método `sendall(bytes)` de socket.

Para recibir un mensaje, se usa el método `recv(buffer)->bytes` de socket.
* `buffer`: cantidad máxima de bytes a recibir.

Para cerrar la conexión se usa el método `close()` de socket.

In [ ]:
import socket

socket_cliente = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
print(socket_cliente)
HOST = socket.gethostname() 
# Permite obtener la dirección IP del host donde se está ejecutando el programa.
# HOST = "localhost"
PORT = 5000

try:
    socket_cliente.connect((HOST, PORT))
    mensaje_a_enviar = 'Mensaje enviado desde cliente a servidor'
    socket_cliente.sendall(mensaje_a_enviar.encode('utf-8'))
    bytes_recibidos = socket_cliente.recv(4096)
    print(f"Mensaje recibido: {bytes_recibidos.decode('utf-8')}")
except ConnectionError as error:
    print('La conexión ha fallado :o')
    print(f'Error: {error}')
finally:  
    socket_cliente.close()

#### Servidor TCP (Transmission Control Protocol)

Se crea un socket de la misma forma que en el cliente, con `socket(family, type)`.

Para enlazar el socket a un puerto se usa el método `bind((host, port))`.
* `host`:  host en el que se está corriendo el servidor.
* `port`: puerto donde se va a escuchar las conexiones.

Para escuchar conexiones se usa el método `listen()`.

Para aceptar conexiones se usa el método `accept()->socket_cliente, address`.
* `socket_cliente`: socket que permite al servidor comunicarse con el cliente que se acepta.
* `address`: tupla con la dirección IP y el puerto del cliente.

Para cerrar el servidor se usa el método `close()` de socket.

In [ ]:
import socket

socket_servidor = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

HOST = socket.gethostname()
print(HOST)
PORT = 5000

socket_servidor.bind((HOST, PORT))
socket_servidor.listen()

while True:
    try:
        socket_cliente, adress = socket_servidor.accept()
        print(f'Conexión aceptada desde: {adress}')
        bytes_recibidos = socket_cliente.recv(4096)
        print(f"Mensaje recibido: {bytes_recibidos.decode('utf-8')}")
        mensaje_a_enviar = 'Mensaje enviado desde servidor a cliente'
        socket_cliente.sendall(mensaje_a_enviar.encode('utf-8'))
    except ConnectionError as error:
        print('La conexión ha fallado :o')
        print(f'Error: {error}')
    finally:
        socket_servidor.close()

## Envío de información: protocolos 🛡️

### ¿Qué es?

Son esquemas de comunicación para la transferencia de información.

### ¿Para qué sirve?

* Envío de grandes cantidades de datos.
* Estandarización del envío de información.
* Orden en la transferencia de datos.
* Garantiza el correcto traspaso de información.

### ¿Qué información puede contener un protocolo?

* Tamaño de los datos enviados.
* Tamaño de los chunks.
* Orden de los bytes.
* Alfabeto de codificación.

### ¿Qué tipos existen?

#### TCP

* Fidelidad del mensaje (handshake).
* Comprobación del establecimiento y mantención de la comunicación.

#### UDP

* Rapidez de tranmisión.
* Facilidad de establecimiento.

**Nota: Siempre se utilizará el protocolo TCP cuando sea relevante la fidelidad del mensaje por sobre la velocidad de transmisión**

### Ejemplo! ✨

In [1]:
import json

TAMANO_CHUNK = 4

mensaje = {
    'nombre': 'Margot',
    'edad': 15,
    'raza': 'Schnauzer'
}

bytes_mensaje = json.dumps(mensaje).encode('utf-8')

while len(bytes_mensaje) % TAMANO_CHUNK != 0:
    bytes_mensaje += b'\x00'

numero_chunks = len(bytes_mensaje) // TAMANO_CHUNK
bytes_numero_chunks = numero_chunks.to_bytes(4, 'little')

mensaje_final = bytes_numero_chunks + bytes_mensaje
print(f'Mensaje final: {mensaje_final}')

Mensaje final: b'\x0e\x00\x00\x00{"nombre": "Margot", "edad": 15, "raza": "Schnauzer"}\x00\x00\x00'


In [2]:
numero_chunks = int.from_bytes(mensaje_final[:4], 'little')

chunks_recibidos = 0
bytes_recibidos = b''
actual = 4

while chunks_recibidos < numero_chunks:
    chunk = mensaje_final[actual:actual+TAMANO_CHUNK]
    bytes_recibidos += chunk
    actual += TAMANO_CHUNK
    chunks_recibidos += 1
print(f'Bytes recibidos: {bytes_recibidos}')

bytes_recibidos_limpios = bytes_recibidos.strip(b"\x00")
mensaje_recibido = json.loads(bytes_recibidos_limpios)
print(f'Mensaje recibido: {mensaje_recibido}')

Bytes recibidos: b'{"nombre": "Margot", "edad": 15, "raza": "Schnauzer"}\x00\x00\x00'
Mensaje recibido: {'nombre': 'Margot', 'edad': 15, 'raza': 'Schnauzer'}


## Manejo de múltiples clientes en forma concurrente 😱

<center><img src="./imagenes/clientes.png"/></center>

* El manejo de varios usuarios simultáneamente se logra mediante el uso de **threads**.
* Un thread encargado de aceptar nuevos clientes y un thread encargado de cada cliente.

### Actividad: 💎 DCCobblestone ⛏

<center><img src="./imagenes/dccobblestone.jpg"/></center>

Tus amigos te han invitado a jugar a un famoso juego de minería: DCCobblestone ⛏. Te parece una buena idea, por lo que decides aceptar. ¿El problema? La página está caída 📉.

Con tus conocimientos de Networking y Sockets decides crear tu propia copia del simulador de minería  (que se juegue a través de la consola… no hay tanto presupuesto 💰). Para ello, deberás escribir un cliente que sea capaz de enviar y recibir mensajes, y un servidor que pueda almacenar información de los jugadores y procesar las acciones que se realizan. 

¡A trabajar 👨‍💻! 


Para que tu simulador resulte en una experiencia divertida, deberás implementar lo siguiente:
* El servidor debe ser capaz de aceptar múltiples jugadores.
* Los mensajes se deben codificar en chunks de largo 60 bytes.
* Se debe codificar en _byteorder big_.
* El servidor debe verificar que los mensajes tengan un largo entre 5 y 60 bytes (sin contar x00), en caso contrario no debe responder.
* El servidor debe verificar que el mensaje sea un comando válido. Este puede ser `"minar"` o `"stats"`.
* Si el mensaje es el comando `"minar"`, el servidor debe actualizar las estadísticas del jugador en cuestión y comunicar qué es lo que se minó a todos los jugadores conectados.
* Si el mensaje es el comando `"stats"`, el servidor debe enviar al jugador un string con los materiales que ha recolectado.
* Si el mensaje es un comando no válido, el servidor tiene que comunicarlo al jugador.


#### Servidor

In [ ]:
import json
import socket
import threading
import random
import sys

class Servidor:

    #Completar
    def __init__(self, host, port):
        pass

In [ ]:
class Miner:
    id = 0
    def __init__(self, socket_cliente):
        self.id_minero = Miner.id
        self.socket = socket_cliente
        self.materiales = {
            "Carbon" : 0,
            "Hierro" : 0,
            "Oro" : 0,
            "Diamante" : 0
        }
        Miner.id += 1

In [ ]:
    #Completar
    def aceptar_clientes(self):
        pass

In [ ]:
    #Completar
    def escuchar_cliente(self, minero):
        pass

In [ ]:
    def recibir(self, socket_cliente):
        largo_bytes_mensaje = socket_cliente.recv(4)
        largo_mensaje = int.from_bytes(largo_bytes_mensaje, byteorder='big')
        bytes_mensaje = bytearray()
        while len(bytes_mensaje) < largo_mensaje:
            bytes_mensaje += socket_cliente.recv(60)
        bytes_mensaje_limpios = bytes_mensaje.strip(b'\x00')
        if 5 < len(bytes_mensaje_limpios) < 60:
            mensaje = self.decodificar_mensaje(bytes_mensaje_limpios)
            return mensaje
        else:
            return ""

In [ ]:
    def enviar(self, mensaje, socket_cliente):
        bytes_mensaje = self.codificar_mensaje(mensaje)
        while len(bytes_mensaje) % 60 != 0:
            bytes_mensaje += b'\x00'
        largo_bytes_mensaje = len(bytes_mensaje).to_bytes(4, byteorder='big')
        socket_cliente.sendall(largo_bytes_mensaje + bytes_mensaje)

In [ ]:
    def enviar_todos(self, mensaje):
        for socket_cliente in self.clientes.values():
            bytes_mensaje = self.codificar_mensaje(mensaje)
            while len(bytes_mensaje) % 60 != 0:
                bytes_mensaje += b'\x00'
            largo_bytes_mensaje = len(bytes_mensaje).to_bytes(4, byteorder='big')
            socket_cliente.sendall(largo_bytes_mensaje + bytes_mensaje)

In [ ]:
    def codificar_mensaje(self, mensaje):
        try:
            mensaje_json = json.dumps(mensaje)
            bytes_mensaje = mensaje_json.encode('utf-8')
            return bytes_mensaje
        except json.JSONDecodeError:
            print('No se pudo codificar el mensaje')
            return b''

In [ ]:
    def decodificar_mensaje(self, bytes_mensaje):
        try:
            mensaje = json.loads(bytes_mensaje)
            return mensaje
        except json.JSONDecodeError:
            print('No se pudo decodificar el mensaje')
            return ''

In [ ]:
    def manejar_mensaje_recibido(self, mensaje, minero):
        if mensaje == "minar":
            material = random.choice(["Carbon", "Hierro", "Oro", "Diamante"])
            cantidad = random.randint(1, 4)
            minero.materiales[material] += cantidad

            return f"Minero #{minero.id_minero} ha minado {cantidad} de {material}"
        elif mensaje == "stats":
            mensaje_stats = f"Estadisticas del minero #{minero.id_minero}"
            for material in minero.materiales:
                mensaje_stats += f"\n> {material} - {minero.materiales[material]}"
                
            return mensaje_stats
            
        return "Comando no válido"

#### Cliente

In [ ]:
class Cliente:

    #Completar
    def __init__(self, host, port):
        pass

In [ ]:
    #Completar
    def recibir_input(self):
        pass

In [ ]:
    #Completar
    def escuchar_servidor(self):
        pass

In [ ]:
    def recibir(self):
        largo_bytes_mensaje = self.socket_cliente.recv(4)
        largo_mensaje = int.from_bytes(largo_bytes_mensaje, byteorder='big')
        bytes_mensaje = bytearray()
        while len(bytes_mensaje) < largo_mensaje:
            bytes_mensaje += self.socket_cliente.recv(60)
        bytes_mensaje_limpios = bytes_mensaje.strip(b'\x00')
        mensaje = self.decodificar_mensaje(bytes_mensaje_limpios)
        return mensaje

In [ ]:
    def enviar(self, mensaje):
        bytes_mensaje = self.codificar_mensaje(mensaje)
        while len(bytes_mensaje) % 60 != 0:
            bytes_mensaje += b'\x00'
        largo_bytes_mensaje = len(bytes_mensaje).to_bytes(4, byteorder='big')
        self.socket_cliente.sendall(largo_bytes_mensaje + bytes_mensaje)

In [ ]:
    def codificar_mensaje(self, mensaje):
        try:
            mensaje_json = json.dumps(mensaje)
            bytes_mensaje = mensaje_json.encode('utf-8')
            return bytes_mensaje
        except json.JSONDecodeError:
            print('No se pudo codificar el mensaje')
            return b''

In [ ]:
    def decodificar_mensaje(self, bytes_mensaje):
        try:
            mensaje = json.loads(bytes_mensaje)
            return mensaje
        except json.JSONDecodeError:
            print('No se pudo decodificar el mensaje')
            return ''

In [ ]:
    def manejar_mensaje_recibido(self, mensaje):
        # Aquí va la lógica de recibir el mensaje, pero en este caso el cliente únicamente imprime la información
        pass